In [2]:
import pandas
import pysdql

path = r'T:\tpch_dataset\100M_with_sql'
answer_path = r'T:\tpch_dataset\100M_with_sql\answers'

customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)
partsupp = pandas.read_table(rf'{path}\partsupp.tbl', sep='|', index_col=False, header=None, names=pysdql.PARTSUPP_COLS)
region = pandas.read_table(rf'{path}\region.tbl', sep='|', index_col=False, header=None, names=pysdql.REGION_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)

In [5]:
# TPCH - 1
'''
l_shipdate <= date '1998-12-01' - interval ':1' day (3)
'''
var1 = '1998-09-05'
r = lineitem[lineitem['l_shipdate'] <= var1]

r['disc_price'] = r['l_extendedprice'] * (1 - r['l_discount'])
r['charge'] = r['l_extendedprice'] * (1 - r['l_discount']) * (1 + r['l_tax'])

r = r.groupby(['l_returnflag', 'l_linestatus'], as_index=False).agg(sum_qty=('l_quantity', 'sum'),
                                                    sum_base_price=('l_extendedprice', 'sum'),
                                                    sum_disc_price=('disc_price', 'sum'),
                                                    sum_charge=('charge', 'sum'),
                                                    avg_qty=('l_quantity', 'mean'),
                                                    avg_price=('l_extendedprice', 'mean'),
                                                    avg_disc=('l_discount', 'mean'),
                                                    count_order=('l_returnflag', 'count')
                                                    )
r

C:\Users\Y\AppData\Local\Temp\ipykernel_22052\3591749755.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['disc_price'] = r['l_extendedprice'] * (1 - r['l_discount'])
C:\Users\Y\AppData\Local\Temp\ipykernel_22052\3591749755.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['charge'] = r['l_extendedprice'] * (1 - r['l_discount']) * (1 + r['l_tax'])


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,3774200,5.320754e+09,5.054096e+09,5.256751e+09,25.537587,36002.123829,0.050145,147790
1,N,F,95257,1.337378e+08,1.271324e+08,1.322863e+08,25.300664,35521.326916,0.049394,3765
2,N,O,7472572,1.053091e+10,1.000393e+10,1.040396e+10,25.545333,36000.398144,0.050095,292522
3,R,F,3785523,5.337951e+09,5.071819e+09,5.274406e+09,25.525944,35994.029214,0.049989,148301


In [4]:
aq1 = pandas.read_table(rf'{answer_path}\q1.out', sep='|')

aq1

,l,l.1,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,37734107.0,5.658655e+10,5.375826e+10,5.590907e+10,25.52,38273.13,0.05,1478493
1,N,F,991417.0,1.487505e+09,1.413082e+09,1.469649e+09,25.52,38284.47,0.05,38854
2,N,O,74476040.0,1.117017e+11,1.061182e+11,1.103670e+11,25.50,38249.12,0.05,2920374
3,R,F,37719753.0,5.656804e+10,5.374129e+10,5.588962e+10,25.51,38250.85,0.05,1478870


In [11]:
# TPCH - 2
'''
p_partkey = ps_partkey
and s_suppkey = ps_suppkey
and s_nationkey = n_nationkey
and n_regionkey = r_regionkey
and r_name = ':3'
'''
var1 = 3
var2 = 'TIN'
var3 = 'AMERICA'
# r = partsupp.merge(supplier, how='inner', left_on='ps_suppkey', right_on='s_suppkey')\
#     .merge(nation, how='inner', left_on='s_nationkey', right_on='n_nationkey')\
#     .merge(region, how='inner', left_on='n_regionkey', right_on='r_regionkey')
r = partsupp.merge(supplier, how='inner', left_on='ps_suppkey', right_on='s_suppkey')\
    .merge(nation, how='inner', left_on='s_nationkey', right_on='n_nationkey')\
    .merge(region, how='inner', left_on='n_regionkey', right_on='r_regionkey')

r = r[(r['r_name'] == var3)].groupby(['ps_partkey'], as_index=False).min()

s = part.merge(partsupp, how='inner', left_on='p_partkey', right_on='ps_partkey')\
    .merge(supplier, how='inner', left_on='ps_suppkey', right_on='s_suppkey')\
    .merge(nation, how='inner', left_on='s_nationkey', right_on='n_nationkey')\
    .merge(region, how='inner', left_on='n_regionkey', right_on='r_regionkey')

s = s[(s['p_size'] == 3)
      & s['p_type'].str.endswith(var2)
      & (s['r_name'] == var3)]

t = s.merge(r, how='inner', left_on='p_partkey', right_on='ps_partkey')
# [['s_acctbal', 's_name', 'n_name', 'p_partkey', 'p_mfgr', 's_address', 's_phone', 's_comment']]
print(t)

    p_partkey                                p_name          p_mfgr   p_brand  \
0        8751         lace light slate smoke yellow  Manufacturer#2  Brand#25   
1        8751         lace light slate smoke yellow  Manufacturer#2  Brand#25   
2       19751  aquamarine dim coral white chocolate  Manufacturer#4  Brand#45   
3       19751  aquamarine dim coral white chocolate  Manufacturer#4  Brand#45   
4        9554    thistle turquoise snow linen olive  Manufacturer#3  Brand#34   
..        ...                                   ...             ...       ...   
60       6314  white turquoise plum goldenrod smoke  Manufacturer#5  Brand#51   
61       9939        yellow deep bisque mint orchid  Manufacturer#2  Brand#21   
62      19671     linen almond misty sienna magenta  Manufacturer#5  Brand#54   
63       6480    floral spring beige black cornsilk  Manufacturer#5  Brand#54   
64       6480    floral spring beige black cornsilk  Manufacturer#5  Brand#54   

                   p_type  